# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# NOTE: the code below couldn't find all 30 files in the event_data directory so I changed the glob expression
file_path_list = glob.glob(os.path.join(filepath, '*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# loop for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# Make a connection to a Cassandra instance my local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# Create a Keyspace . Using class  
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Setting KEYSPACE
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [12]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## For table music_play_session_history, the column_a was used as a partition key because the queries will filter by this column. 
## session_id and item_in_session were used as clustering columns to help make up a unique key.

query = """CREATE TABLE IF NOT EXISTS music_play_session_history (
    session_id INT, 
    item_in_session INT,
    artist_name TEXT, 
    song_title TEXT, 
    song_length FLOAT,
    PRIMARY KEY (session_id, item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [13]:
# csv file is assigned to file variable.
file = 'event_datafile_new.csv'
# read data with data loop
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements are assigned to the query variable
        query = "INSERT INTO music_play_session_history (session_id, item_in_session, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Sets which column element to assign for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## The SELECT statement is used to verify that the data has been entered into the table.
query = "SELECT artist_name, song_title, song_length FROM music_play_session_history WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## For table music_play_user_sessions, the column_a was used as a partition key because the queries will filter by this column. 
## user_id and session_id were used as clustering columns to help make up a unique key.

query = """CREATE TABLE IF NOT EXISTS music_play_user_sessions (
    user_id INT,
    session_id INT, 
    item_in_session INT,
    artist_name TEXT, 
    song_title TEXT, 
    user_firstname TEXT,
    user_lastname TEXT, 
    PRIMARY KEY ((user_id, session_id), item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)        

                    

In [16]:
# The table is populated using the csv file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_play_user_sessions (user_id, session_id, item_in_session, artist_name, song_title, user_firstname, user_lastname) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [17]:
# Query table to validate data
query = "SELECT artist_name, song_title, user_firstname, user_lastname FROM music_play_user_sessions WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## For table song_paly_user_history, the column_a was used as a partition key because the queries will filter by this column. 
## song_title and user_id were used as clustering columns to help make up a unique key.

query = """CREATE TABLE IF NOT EXISTS song_paly_user_history (
    song_title TEXT, 
    user_id INT,
    user_firstname TEXT,
    user_lastname TEXT,
    PRIMARY KEY (song_title, user_id))"""
try:
    session.execute(query)
except Exception as e:
    print(e)        
                    

In [19]:
# Populate table using csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_paly_user_history (song_title, user_id, user_firstname, user_lastname) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [20]:
# Query table to verify the data
query = "SELECT user_firstname, user_lastname FROM song_paly_user_history WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_firstname, row.user_lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
## Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS music_play_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [22]:
query = "DROP TABLE IF EXISTS music_play_user_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [23]:
query = "DROP TABLE IF EXISTS song_paly_user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
## Close the session and cluster.
session.shutdown()
cluster.shutdown()